In [1]:
!pip install autogen-agentchat~=0.2 autogen psutil --quiet 
# !pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client==0.12.3 langgraph langchain_experimental yfinance
!pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet
!pip install -U docker --quiet 


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import autogen
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated
from typing import List, Optional
from datetime import datetime, timedelta
from dataclasses import dataclass
from autogen import ConversableAgent
from dotenv import load_dotenv

import autogen
from autogen.cache import Cache

DEEPSEEK_INFERENCE_SERVER_URL = "http://localhost:8989"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
DEEPSEEK_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

INFERENCE_SERVER_URL = "http://localhost:8000"
MODEL_NAME = "ibm-granite/granite-3.1-8b-base"
API_KEY= "alanliuxiang"


In [3]:
# Configuration for the vLLM endpoint
llm_config1 = {
    "config_list": [
        {
            "model": MODEL_NAME,
            "api_key": API_KEY,
            "base_url": f"{INFERENCE_SERVER_URL}/v1"
        }
    ],
    "cache_seed": None,
    "temperature": 0.01,
    "timeout": 600,
}

llm_config = {
    "config_list": [
        {
            "model": DEEPSEEK_MODEL_NAME,
            "api_key": API_KEY,
            "base_url": f"{DEEPSEEK_INFERENCE_SERVER_URL}/v1"
        }
    ],
    "cache_seed": None,
    "temperature": 0.01,
    "timeout": 600,
}

In [4]:
from dataclasses import dataclass
from datetime import datetime, timedelta
import os
from typing import List, Optional
from dotenv import load_dotenv
from autogen import (
    ConversableAgent,
    UserProxyAgent,
    GroupChat,
    GroupChatManager,
)

In [6]:

@dataclass
class FlightDetails:
    flight_number: str
    status: str
    departure: datetime
    arrival: datetime
    price: float
    seats_available: int

    def to_dict(self):
        return {
            "flight_number": self.flight_number,
            "status": self.status,
            "departure": self.departure.isoformat(),
            "arrival": self.arrival.isoformat(),
            "price": self.price,
            "seats_available": self.seats_available,
        }


@dataclass
class HotelDetails:
    name: str
    location: str
    price: float
    rating: float
    reviews: List[str]
    available_rooms: int

    def to_dict(self):
        return {
            "name": self.name,
            "location": self.location,
            "price": self.price,
            "rating": self.rating,
            "reviews": self.reviews,
            "available_rooms": self.available_rooms,
        }


@dataclass
class LocationInfo:
    weather: str
    events: List[str]
    safety_alerts: List[str]
    local_time: datetime

    def to_dict(self):
        return {
            "weather": self.weather,
            "events": self.events,
            "safety_alerts": self.safety_alerts,
            "local_time": self.local_time.isoformat(),
        }


In [7]:
class TravelTools:
    @staticmethod
    def get_flight_status(flight_number: str, date: Optional[str] = None) -> dict:
        return FlightDetails(
            flight_number=flight_number,
            status="On Time",
            departure=datetime.now(),
            arrival=datetime.now() + timedelta(hours=2),
            price=299.99,
            seats_available=15,
        ).to_dict()

    @staticmethod
    def track_flight_prices(origin: str, destination: str, date_range: str) -> dict:
        return {
            "price_history": [320.0, 310.0, 299.99],
            "price_forecast": [305.0, 315.0, 325.0],
        }

    @staticmethod
    def get_hotel_details(location: str, check_in: str, check_out: str) -> dict:
        return HotelDetails(
            name="Grand Hotel",
            location=location,
            price=199.99,
            rating=4.5,
            reviews=["Great location", "Excellent service"],
            available_rooms=5,
        ).to_dict()

    @staticmethod
    def get_location_info(location: str, date: Optional[str] = None) -> dict:
        return LocationInfo(
            weather="Sunny, 75°F",
            events=["Local Festival", "Art Exhibition"],
            safety_alerts=["No current alerts"],
            local_time=datetime.now(),
        ).to_dict()


def check_termination(msg):
    try:
        content = msg.get("content", "")
        if isinstance(content, str):
            if "TERMINATE" in content or any(
                term in content.lower()
                for term in ["completed", "here are the results", "finished"]
            ):
                return True
        return False
    except Exception:
        return False

In [8]:
class TravelAgentSystem:
    def __init__(self, llm_config: dict):

        self.tools = TravelTools()

        self.travel_assistant = ReActAgent(
            name="TravelAssistant",
            system_message="You plan travel using a systematic approach.",
            llm_config=llm_config,
        )

        self.user_proxy = ConversableAgent(
            name="UserProxy",
            is_termination_msg=check_termination,
            human_input_mode="NEVER",
        )

        self._register_tools()

    def _register_tools(self):
        tools = [
            (self.tools.get_flight_status, "Get current Flight Status"),
            (self.tools.track_flight_prices, "Track Flight Prices"),
            (self.tools.get_hotel_details, "Get Hotel Details"),
            (self.tools.get_location_info, "Get Location Info"),
        ]

        for tool, description in tools:
            # Register the function with its description for travel_assistant
            self.travel_assistant.register_for_llm(
                name=tool.__name__,
                description=description,
            )(tool)

            self.user_proxy.register_for_execution(name=tool.__name__)(tool)

    def run_query(self, query: str):
        return self.user_proxy.initiate_chat(self.travel_assistant, message=query)


def main():
    travel_system = TravelAgentSystem(llm_config)

    # Test case demonstrating the ReAct pattern
    queries = [
        "Plan a trip to NYC: need flight AA123 status, hotel for next week, and local events",
        "Find the cheapes time to fly from SFO to NYC next month and suggest a hotels",
    ]

    for query in queries:
        print(f"\nUser Query: {query}")
        result = travel_system.run_query(query)
        print("-" * 50)
        # print(f"Agent Response: \n\n{result}")

In [9]:
# if __name__ == "__main__":
main()


User Query: Plan a trip to NYC: need flight AA123 status, hotel for next week, and local events
UserProxy (to TravelAssistant):

Plan a trip to NYC: need flight AA123 status, hotel for next week, and local events

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 03-15 05:15:31] {652} WARNING - Model deepseek-ai/DeepSeek-R1-Distill-Qwen-7B is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
TravelAssistant (to UserProxy):

Okay, so I need to plan a trip to NYC. The user mentioned three main things: checking the status of flight AA123, booking a hotel for the next week, and finding local events. Let me break this down step by step.

First, checking the flight status. I remember that FlightAware is a tool that can track flights in real-time. I should use that. I'll need to input the flight number, which is